## Training an RL agent to predict country capitals

This notebook presents a worked example of how to build an RL agent with Kandula. In this example we build an RL agent that learns what are the capitals of different countrys.

### Setup

First things first, let's import the requirements. 

In [3]:
from kandula import logging
from kandula.steps import RLStep
from kandula.qtable import QTable
from kandula.Q_learning import QL
from typing import List

from functools import reduce
import torch
import json
import random
from nltk import word_tokenize

For this example, in order to train the RL agent, we have downloaded a json file that contains the countris of the world and their capitals from [this repository](https://github.com/icyrockcom/country-capitals/blob/master/data/country-list-with-ids.json). For convenience, we downloaded the file into the [presources folder](https://github.com/meghdadFar/kandula/tree/main/resources) in this repository. Let's read this file and prune it so that it suits our needs.

In [4]:
with open("../resources/country_capital.json", "r") as fc:
    capitals: List = json.load(fc)
capitals_dict = {}
country_index = {}
index_country = {}
i=1
for jl in capitals:
    capitals_dict[jl["country"]] = jl["capital"]
    country_index[jl["country"]] = i
    index_country[i] = jl["country"]
    i+=1

In the above code, we first read the json lines into `capitals`, we then create three dictionaries from it: `capitals_dict` that maps the country names to their capitals, `country_index` that maps the country names to incremental indexes, and `index_country` that maps back indexes to country names.

### Define the State Space

The first thing that we should consider is how do we want to map our problem to a state-action space. In this example, I consider that each contry represents a stae and hence, for 248 countries we will have a 1-dimensional state space of 248. If the space was 2-dimensional and say the first dimension had a size N and the second dimension has a size M, we could have defined the `state_space` variable as: `[N, M]`

In [5]:
state_space = [248]

### Define the Actions

The next thing is to define the actions. In this example, I consider guessing (the right) capital as an action that my RL agent is suppose to learn and hence, I define my actions to be a list of 248 capitals. If for instance, my RL agent was supposed to take one of the two actions of e.g. shifting grear up or shifting gear down, my actions variable would have been `[shift_gear_up, shift_gear_down]`.

In [ ]:
actions = [v for _, v in capitals_dict.items()]

### Define the RL Step

It's now time to define our RL step. An RL step should be a child of `kandula.steps.RLStep` class and implement its two abstract methods, namely `get_state()` and `get_reward()`. In addition to the definition of states and actions, this is where you make the RL agent really specific to your problem.

In [6]:
class CapitalsRLStep(RLStep):

    def get_state(self):
        country = gen_rand_country()
        state = [country_index[country]]
        return state
    
    def get_reward(self, state, action):
        s = reduce((lambda x: x), state)
        reward = 1 if capitals_dict[index_country[s]] == action else 0
        return reward

In the above code, the state situation is simplified which matches this particular problem. To get the current state we simply choose a random country (ignoring the previous action and the possible changes of the enviroment) via `gen_rand_country()`. The reward is calculated by comparing the RL agent's prediction to the actual capital.

### Initiate and Train the RL Agent

In [ ]:
mrls = CapitalsRLStep()
qt = QTable(state_space=state_space, actions=actions)
ql = QL(qtable=qt, rl_step=mrls)
ql.train(3000000, get_correct_action_for_capitals)

Before being able to train our RL agent, we need to define one more function that is used in the training loop and its main purpose is gather how the RL agent is doing during the training.